# Laplacian Convolutional Representation (LCR)

In [1]:
import numpy as np

def compute_mape(var, var_hat):
    return np.sum(np.abs(var - var_hat) / var) / var.shape[0]

def compute_rmse(var, var_hat):
    return np.sqrt(np.sum((var - var_hat) ** 2) / var.shape[0])

def laplacian(T, tau):
    ell = np.zeros(T)
    ell[0] = 2 * tau
    for k in range(tau):
        ell[k + 1] = -1
        ell[-k - 1] = -1
    return ell

def prox_2d(z, w, lmbda, denominator):
    N, T = z.shape
    temp = np.fft.fft2(lmbda * z - w) / denominator
    temp1 = 1 - N * T / (lmbda * np.abs(temp))
    temp1[temp1 <= 0] = 0
    return np.fft.ifft2(temp * temp1).real

def update_z(y_train, pos_train, x, w, lmbda, eta):
    z = x + w / lmbda
    z[pos_train] = (lmbda / (lmbda + eta) * z[pos_train] 
                    + eta / (lmbda + eta) * y_train)
    return z

def update_w(x, z, w, lmbda):
    return w + lmbda * (x - z)

def LCR_2d(y_true, y, lmbda, gamma, tau, maxiter = 50):
    eta = 100 * lmbda
    N, T = y.shape
    pos_train = np.where(y != 0)
    y_train = y[pos_train]
    pos_test = np.where((y_true != 0) & (y == 0))
    y_test = y_true[pos_test]
    z = y.copy()
    w = y.copy()
    ell_s = np.zeros(N)
    ell_s[0] = 1
    ell_t = laplacian(T, tau)
    denominator = lmbda + gamma * np.fft.fft2(np.outer(ell_s, ell_t)) ** 2
    del y_true, y
    show_iter = 20
    for it in range(maxiter):
        x = prox_2d(z, w, lmbda, denominator)
        z = update_z(y_train, pos_train, x, w, lmbda, eta)
        w = update_w(x, z, w, lmbda)
        if (it + 1) % show_iter == 0:
            print(it + 1)
            print(compute_mape(y_test, x[pos_test]))
            print(compute_rmse(y_test, x[pos_test]))
            print()
    return x

In [2]:
import numpy as np
np.random.seed(1000)

dense_mat = np.load('../datasets/California-data-set/pems-w1.npz')['arr_0']
for t in range(2, 5):
    dense_mat = np.append(dense_mat, np.load('../datasets/California-data-set/pems-w{}.npz'.format(t))['arr_0'], 
                          axis = 1)
dim1, dim2 = dense_mat.shape

missing_rate = 0.3
sparse_mat = dense_mat * np.round(np.random.rand(dim1, dim2) + 0.5 - missing_rate)

import time
start = time.time()
N, T = sparse_mat.shape
lmbda = 1e-5 * N * T
gamma = 5 * lmbda
tau = 1
maxiter = 100
x = LCR_2d(dense_mat, sparse_mat, lmbda, gamma, tau, maxiter)
end = time.time()
print('Running time: %d seconds.'%(end - start))

20
0.016843134726602475
1.5953675937925527

40
0.016103390820203022
1.5579640806657191

60
0.01587799815456933
1.5471327544825788

80
0.01575579473524034
1.5414270110601587

100
0.015678064028916847
1.537766795152083

Running time: 1747 seconds.


In [3]:
import numpy as np
np.random.seed(1000)

dense_mat = np.load('../datasets/California-data-set/pems-w1.npz')['arr_0']
for t in range(2, 5):
    dense_mat = np.append(dense_mat, np.load('../datasets/California-data-set/pems-w{}.npz'.format(t))['arr_0'], 
                          axis = 1)
dim1, dim2 = dense_mat.shape

missing_rate = 0.5
sparse_mat = dense_mat * np.round(np.random.rand(dim1, dim2) + 0.5 - missing_rate)

import time
start = time.time()
N, T = sparse_mat.shape
lmbda = 1e-5 * N * T
gamma = 5 * lmbda
tau = 1
maxiter = 100
x = LCR_2d(dense_mat, sparse_mat, lmbda, gamma, tau, maxiter)
end = time.time()
print('Running time: %d seconds.'%(end - start))

20
0.01947881038739679
1.7943322571037228

40
0.018360177873393866
1.7501990577366606

60
0.018179060311325112
1.745680629400671

80
0.018101260247569837
1.744165026625778

100
0.018054818060158857
1.7432085734776304

Running time: 1719 seconds.


In [4]:
import numpy as np
np.random.seed(1000)

dense_mat = np.load('../datasets/California-data-set/pems-w1.npz')['arr_0']
for t in range(2, 5):
    dense_mat = np.append(dense_mat, np.load('../datasets/California-data-set/pems-w{}.npz'.format(t))['arr_0'], 
                          axis = 1)
dim1, dim2 = dense_mat.shape

missing_rate = 0.7
sparse_mat = dense_mat * np.round(np.random.rand(dim1, dim2) + 0.5 - missing_rate)

import time
start = time.time()
N, T = sparse_mat.shape
lmbda = 1e-5 * N * T
gamma = 5 * lmbda
tau = 1
maxiter = 100
x = LCR_2d(dense_mat, sparse_mat, lmbda, gamma, tau, maxiter)
end = time.time()
print('Running time: %d seconds.'%(end - start))

20
0.027327014480528284
2.309207186585469

40
0.02316442146220143
2.121940638698902

60
0.022952203182093216
2.1201777424887274

80
0.0229116617027545
2.1219363360946257

100
0.022896774859259294
2.123219654110104

Running time: 1698 seconds.


In [5]:
import numpy as np
np.random.seed(1000)

dense_mat = np.load('../datasets/California-data-set/pems-w1.npz')['arr_0']
for t in range(2, 5):
    dense_mat = np.append(dense_mat, np.load('../datasets/California-data-set/pems-w{}.npz'.format(t))['arr_0'], 
                          axis = 1)
dim1, dim2 = dense_mat.shape

missing_rate = 0.9
sparse_mat = dense_mat * np.round(np.random.rand(dim1, dim2) + 0.5 - missing_rate)

import time
start = time.time()
N, T = sparse_mat.shape
lmbda = 1e-5 * N * T
gamma = 5 * lmbda
tau = 1
maxiter = 100
x = LCR_2d(dense_mat, sparse_mat, lmbda, gamma, tau, maxiter)
end = time.time()
print('Running time: %d seconds.'%(end - start))

20
0.052736772532630094
3.909264762107882

40
0.04065298085132097
3.2815492608085433

60
0.03847692679641931
3.1886413484393614

80
0.038094864783371804
3.171354772458791

100
0.038033600385423263
3.169204924646335

Running time: 1680 seconds.


### License

<div class="alert alert-block alert-danger">
<b>This work is released under the MIT license.</b>
</div>